Here I will explore the data encoding that will need to take place to be able to create our baseline model

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


In [28]:
# put the clean data into a dataframe
df = pd.read_csv('../raw_data/match_result_averages.csv')
df.head()

,ID,Team_x,Team_batting_average,WinningTeam,Team_y,Team_batting_average_B
0,1312200,Rajasthan Royals,155.397906,Gujarat Titans,Gujarat Titans,166.437500
1,1312199,Royal Challengers Bangalore,155.707965,Rajasthan Royals,Rajasthan Royals,155.397906
2,1312198,Royal Challengers Bangalore,155.707965,Royal Challengers Bangalore,Lucknow Super Giants,169.866667
3,1312197,Rajasthan Royals,155.397906,Gujarat Titans,Gujarat Titans,166.437500
4,1304116,Sunrisers Hyderabad,155.546053,Punjab Kings,Punjab Kings,160.464286


In [29]:
# Renames the columns for readability
df.rename(columns={'Team_batting_average': 'team_x_batting_average'}, inplace=True)
df.rename(columns={'Team_batting_average_B': 'team_y_batting_average'}, inplace=True)

In [30]:
# Create an initiation of the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False) 

In [31]:
# Fit and transform the encoder on the specified columns
encoded_columns = encoder.fit_transform(df[['Team_x', 'Team_y']])

In [32]:
# Create a DataFrame with the one-hot encoded columns
encoded_df = pd.DataFrame(encoded_columns, columns=encoder.get_feature_names_out(['Team_x', 'Team_y']))

In [33]:
# Concatenate the encoded DataFrame with the original DataFrame
df_encoded = pd.concat([df, encoded_df], axis=1)

In [34]:
# Drop the original columns as they are no longer needed
df_encoded.drop(['Team_x', 'Team_y'], axis=1, inplace=True)
df_encoded.shape

(948, 40)

In [36]:
df_encoded.head(10)

,ID,team_x_batting_average,WinningTeam,team_y_batting_average,Team_x_Chennai Super Kings,Team_x_Deccan Chargers,Team_x_Delhi Capitals,Team_x_Delhi Daredevils,Team_x_Gujarat Lions,Team_x_Gujarat Titans,...,Team_y_Kolkata Knight Riders,Team_y_Lucknow Super Giants,Team_y_Mumbai Indians,Team_y_Pune Warriors,Team_y_Punjab Kings,Team_y_Rajasthan Royals,Team_y_Rising Pune Supergiant,Team_y_Rising Pune Supergiants,Team_y_Royal Challengers Bangalore,Team_y_Sunrisers Hyderabad
0,1312200,155.397906,Gujarat Titans,166.437500,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1312199,155.707965,Rajasthan Royals,155.397906,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1312198,155.707965,Royal Challengers Bangalore,169.866667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1312197,155.397906,Gujarat Titans,166.437500,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1304116,155.546053,Punjab Kings,160.464286,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,1304115,161.031746,Mumbai Indians,158.714286,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1304114,160.543269,Rajasthan Royals,155.397906,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,1304113,166.437500,Royal Challengers Bangalore,155.707965,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,1304112,169.866667,Lucknow Super Giants,153.367713,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1304111,155.546053,Sunrisers Hyderabad,158.714286,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder for the target variable
label_encoder = LabelEncoder()

# Encode the 'WinningTeam' column
df_encoded['WinningTeam_encoded'] = label_encoder.fit_transform(df_encoded['WinningTeam'])

In [41]:
# Define features (X) and the encoded target variable (y)
X = df_encoded.drop(['WinningTeam', 'WinningTeam_encoded', 'ID'], axis=1)
y = df_encoded['WinningTeam_encoded']

In [42]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [56]:
# Initialize and train the logistic regression model
model = LogisticRegression(max_iter=2000)
model.fit(X_train, y_train)

/Users/patrickevans29/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=2000)

In [57]:
# Make predictions on the testing set
y_pred = model.predict(X_test)

In [59]:
# Evaluate the model
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')

Accuracy: 0.5631578947368421


## Train a new model using all label encoded targets and ignoring average

In [76]:
# Create an initiation of the OneHotEncoder
encoder2 = LabelEncoder()

In [77]:
# Fit and transform the encoder on the specified columns
encoded_columns2 = encoder2.fit_transform(df[['Team_x', 'Team_y', 'WinningTeam']])

ValueError: y should be a 1d array, got an array of shape (948, 3) instead.

In [64]:
# Create a DataFrame with the one-hot encoded columns
encoded_df2 = pd.DataFrame(encoded_columns2, columns=encoder2.get_feature_names_out(['Team_x', 'Team_y', 'WinningTeam']))

In [65]:
# Concatenate the encoded DataFrame with the original DataFrame
df_encoded2 = pd.concat([df, encoded_df2], axis=1)

In [67]:
# Drop the original columns as they are no longer needed
df_encoded2.drop(['Team_x', 'Team_y'], axis=1, inplace=True)

In [69]:
df_encoded2.columns

Index(['ID', 'team_x_batting_average', 'WinningTeam', 'team_y_batting_average',
       'Team_x_Chennai Super Kings', 'Team_x_Deccan Chargers',
       'Team_x_Delhi Capitals', 'Team_x_Delhi Daredevils',
       'Team_x_Gujarat Lions', 'Team_x_Gujarat Titans',
       'Team_x_Kings XI Punjab', 'Team_x_Kochi Tuskers Kerala',
       'Team_x_Kolkata Knight Riders', 'Team_x_Lucknow Super Giants',
       'Team_x_Mumbai Indians', 'Team_x_Pune Warriors', 'Team_x_Punjab Kings',
       'Team_x_Rajasthan Royals', 'Team_x_Rising Pune Supergiant',
       'Team_x_Rising Pune Supergiants', 'Team_x_Royal Challengers Bangalore',
       'Team_x_Sunrisers Hyderabad', 'Team_y_Chennai Super Kings',
       'Team_y_Deccan Chargers', 'Team_y_Delhi Capitals',
       'Team_y_Delhi Daredevils', 'Team_y_Gujarat Lions',
       'Team_y_Gujarat Titans', 'Team_y_Kings XI Punjab',
       'Team_y_Kochi Tuskers Kerala', 'Team_y_Kolkata Knight Riders',
       'Team_y_Lucknow Super Giants', 'Team_y_Mumbai Indians',
       'T

In [73]:
# Define features (X) and the encoded target variable (y)
X2 = df_encoded2[['Team_x_Chennai Super Kings', 'Team_x_Deccan Chargers',
       'Team_x_Delhi Capitals', 'Team_x_Delhi Daredevils',
       'Team_x_Gujarat Lions', 'Team_x_Gujarat Titans',
       'Team_x_Kings XI Punjab', 'Team_x_Kochi Tuskers Kerala',
       'Team_x_Kolkata Knight Riders', 'Team_x_Lucknow Super Giants',
       'Team_x_Mumbai Indians', 'Team_x_Pune Warriors', 'Team_x_Punjab Kings',
       'Team_x_Rajasthan Royals', 'Team_x_Rising Pune Supergiant',
       'Team_x_Rising Pune Supergiants', 'Team_x_Royal Challengers Bangalore',
       'Team_x_Sunrisers Hyderabad', 'Team_y_Chennai Super Kings',
       'Team_y_Deccan Chargers', 'Team_y_Delhi Capitals',
       'Team_y_Delhi Daredevils', 'Team_y_Gujarat Lions',
       'Team_y_Gujarat Titans', 'Team_y_Kings XI Punjab',
       'Team_y_Kochi Tuskers Kerala', 'Team_y_Kolkata Knight Riders',
       'Team_y_Lucknow Super Giants', 'Team_y_Mumbai Indians',
       'Team_y_Pune Warriors', 'Team_y_Punjab Kings',
       'Team_y_Rajasthan Royals', 'Team_y_Rising Pune Supergiant',
       'Team_y_Rising Pune Supergiants', 'Team_y_Royal Challengers Bangalore',
       'Team_y_Sunrisers Hyderabad']]

y2 = df_encoded2[['WinningTeam_Deccan Chargers', 'WinningTeam_Delhi Capitals',
       'WinningTeam_Delhi Daredevils', 'WinningTeam_Gujarat Lions',
       'WinningTeam_Gujarat Titans', 'WinningTeam_Kings XI Punjab',
       'WinningTeam_Kochi Tuskers Kerala', 'WinningTeam_Kolkata Knight Riders',
       'WinningTeam_Lucknow Super Giants', 'WinningTeam_Mumbai Indians',
       'WinningTeam_Pune Warriors', 'WinningTeam_Punjab Kings',
       'WinningTeam_Rajasthan Royals', 'WinningTeam_Rising Pune Supergiant',
       'WinningTeam_Rising Pune Supergiants',
       'WinningTeam_Royal Challengers Bangalore',
       'WinningTeam_Sunrisers Hyderabad', 'WinningTeam_nan']]

In [74]:
# Split data into training and testing sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)

In [75]:
# Initialize and train the logistic regression model
model = LogisticRegression(max_iter=2000)
model.fit(X_train2, y_train2)

ValueError: y should be a 1d array, got an array of shape (758, 18) instead.